# Setup

In [ ]:
%pip install graphdatascience

In [ ]:
# Import the client
from graphdatascience import GraphDataScience

# Replace with the actual URI, username, and password
AURA_CONNECTION_URI = "neo4j+s://xxxxxxxx.databases.neo4j.io"
AURA_USERNAME = "neo4j"
AURA_PASSWORD = ""

# Configure the client with AuraDS-recommended settings
gds = GraphDataScience(
    AURA_CONNECTION_URI,
    auth=(AURA_USERNAME, AURA_PASSWORD),
    aura_ds=True
)

In [ ]:
gds.run_cypher("""
    MERGE ( dan:Person:ExampleData   {name: 'Dan',   age: 20, heightAndWeight: [185, 75]})
    MERGE ( annie:Person:ExampleData {name: 'Annie', age: 12, heightAndWeight: [124, 42]})
    MERGE ( matt:Person:ExampleData  {name: 'Matt',  age: 67, heightAndWeight: [170, 80]})
    MERGE ( jeff:Person:ExampleData  {name: 'Jeff',  age: 45, heightAndWeight: [192, 85]})
    MERGE ( brie:Person:ExampleData  {name: 'Brie',  age: 27, heightAndWeight: [176, 57]})
    MERGE ( elsa:Person:ExampleData  {name: 'Elsa',  age: 32, heightAndWeight: [158, 55]})
    MERGE ( john:Person:ExampleData  {name: 'John',  age: 35, heightAndWeight: [172, 76]})

    MERGE (dan)-[:KNOWS {relWeight: 1.0}]->(annie)
    MERGE (dan)-[:KNOWS {relWeight: 1.6}]->(matt)
    MERGE (annie)-[:KNOWS {relWeight: 0.1}]->(matt)
    MERGE (annie)-[:KNOWS {relWeight: 3.0}]->(jeff)
    MERGE (annie)-[:KNOWS {relWeight: 1.2}]->(brie)
    MERGE (matt)-[:KNOWS {relWeight: 10.0}]->(brie)
    MERGE (brie)-[:KNOWS {relWeight: 1.0}]->(elsa)
    MERGE (brie)-[:KNOWS {relWeight: 2.2}]->(jeff)
    MERGE (john)-[:KNOWS {relWeight: 5.0}]->(jeff)

    RETURN True AS exampleDataCreated
""")

In [ ]:
g, result = gds.graph.project(
    "example_graph_for_graphsage",
    {
        "Person": {
            "label": "ExampleData",
            "properties": ["age", "heightAndWeight"]
        }
    },
    {
        "KNOWS": {
            "type": "KNOWS",
            "orientation": "UNDIRECTED",
            "properties": ["relWeight"]
        }
    }
)

print(result)

nodeProjection            {'Person': {'label': 'ExampleData', 'propertie...
relationshipProjection    {'KNOWS': {'orientation': 'UNDIRECTED', 'aggre...
graphName                                       example_graph_for_graphsage
nodeCount                                                                 7
relationshipCount                                                        18
projectMillis                                                            48
Name: 0, dtype: object


# Train a model

In [ ]:
model, result = gds.beta.graphSage.train(
    g,
    modelName="example_graph_model_for_graphsage",
    featureProperties=["age", "heightAndWeight"],
    aggregator="mean",
    activationFunction="sigmoid",
    sampleSizes=[25, 10]
)

# View the model catalog

In [ ]:
result = gds.beta.model.list()

print(result)

                                           modelInfo  \
0  {'modelName': 'example_graph_model_for_graphsa...   

                                         trainConfig  \
0  {'maxIterations': 10, 'negativeSampleWeight': ...   

                                         graphSchema  loaded  stored  \
0  {'relationships': {'KNOWS': {}}, 'nodes': {'Pe...    True   False   

                          creationTime  shared  
0  2022-03-29T15:00:50.485465000+00:00   False  


# Save a model to disk

In [ ]:
result = gds.alpha.model.store(model)

print(result)

modelName      example_graph_model_for_graphsage
storeMillis                                   25
Name: 0, dtype: object


In [ ]:
result = gds.beta.model.list()

print(result)

                                           modelInfo  \
0  {'modelName': 'example_graph_model_for_graphsa...   

                                         trainConfig  \
0  {'maxIterations': 10, 'negativeSampleWeight': ...   

                                         graphSchema  loaded  stored  \
0  {'relationships': {'__ALL__': {}}, 'nodes': {'...    True    True   

                          creationTime  shared  
0  2022-03-29T15:00:50.485465000+00:00   False  


# Share a model with other users

## Create a new user

In [ ]:
# Switch to the "system" database to run the
# "CREATE USER" admin command
gds.set_database("system")

gds.run_cypher("""
    CREATE USER testUser IF NOT EXISTS
    SET PASSWORD 'password'
    SET PASSWORD CHANGE NOT REQUIRED
""")

""


## Publish the model

In [ ]:
# Switch back to the default "neo4j" database
# to publish the model
gds.set_database("neo4j")

model_public = gds.alpha.model.publish(model)

print(model_public)

## View the model as a different user

In [ ]:
test_user_gds = GraphDataScience(
    AURA_CONNECTION_URI,
    auth=("testUser", "password"),
    aura_ds=True
)

result = test_user_gds.beta.model.list()

print(result)

                                           modelInfo  \
0  {'modelName': 'example_graph_model_for_graphsa...   

                                         trainConfig  \
0  {'maxIterations': 10, 'negativeSampleWeight': ...   

                                         graphSchema  loaded  stored  \
0  {'relationships': {'__ALL__': {}}, 'nodes': {'...    True    True   

                          creationTime  shared  
0  2022-03-29T15:00:50.485465000+00:00    True  


# Cleanup

In [ ]:
# Delete the example dataset
gds.run_cypher("""
    MATCH (example:ExampleData)
    DETACH DELETE example
""")

# Delete the projected graph from memory
gds.graph.drop(g)

# Drop the model from memory
gds.beta.model.drop(model_public)

# Delete the model from disk
gds.alpha.model.delete(model_public)

# Switch to the system database and delete the example user
gds.set_database("system")

gds.run_cypher("""
    DROP USER testUser
""")

""
